In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import sys

# Day 8

# Data Loading, Storage, and File Formats
### Reading and Writing Data in Text Format

Python has become a beloved language for text and file munging due to its simple syntax
for interacting with files, intuitive data structures, and convenient features like tuple
packing and unpacking.

pandas features a number of functions for reading tabular data as a DataFrame object

In [227]:
# Parsing functions in pandas

# FUNCTION        DESCRIPTION
# read_csv        Load delimited data from a file, URL, or file-like object. Use comma as default delimiter
# read_table      Load delimited data from a file, URL, or file-like object. Use tab ('\t') as default delimiter
# read_fwf        Read data in fixed-width column format (that is, no delimiters)
# read_clipboard  Version of read_table that reads data from the clipboard. Useful for converting tables from web pages

Let’s start with a small comma-separated (CSV) text file:

In [323]:
!cat C:/srdev/exc1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [106]:
# Since this is comma-delimited, we can use read_csv to read it into a DataFrame:
df = pd.read_csv('C:/srdev/exc1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [114]:
df = pd.read_csv('C:\srdev\exc1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [238]:
# We could also have used read_table and specifying the delimiter:
pd.read_table('C:/srdev/exc1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [115]:
# A file will not always have a header row. Consider this file:
!cat C:/srdev/exc2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [240]:
# To read this in, you have a couple of options. You can allow pandas to assign default column names, 
# or you can specify names yourself:

pd.read_csv('C:/srdev/exc2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [241]:
pd.read_csv('C:/srdev/exc2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [242]:
# Suppose you wanted the message column to be the index of the returned DataFrame.
# You can either indicate you want the column at index 4 or named 'message' using the
# index_col argument:

names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('C:/srdev/exc2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [243]:
# In the event that you want to form a hierarchical index from multiple columns, just
# pass a list of column numbers or names:

!cat C:/srdev/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [245]:
parsed = pd.read_csv('C:/srdev/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [246]:
# In some cases, a table might not have a fixed delimiter, using whitespace or some other
# pattern to separate fields. In these cases, you can pass a regular expression as a delimiter
# for read_table. Consider a text file that looks like this:

list(open('C:/srdev/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [248]:
# While you could do some munging by hand, in this case fields are separated by a variable
# amount of whitespace. This can be expressed by the regular expression \s+, so we have then:

result = pd.read_table('C:/srdev/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [249]:
# Because there was one fewer column name than the number of data rows, read_table
# infers that the first column should be the DataFrame’s index in this special case

!cat C:/srdev/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [250]:
pd.read_csv('C:/srdev/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [252]:
# Handling missing values is an important and frequently nuanced part of the file parsing
# process. Missing data is usually either not present (empty string) or marked by some
# sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as
# NA, -1.#IND, and NULL:

!cat C:/srdev/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [254]:
result = pd.read_csv('C:/srdev/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [255]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [91]:
# The na_values option can take either a list or set of strings to consider missing values

result = pd.read_csv('C:/srdev/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [92]:
# Different NA sentinels can be specified for each column in a dict:

sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('C:/srdev/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [ ]:
# read_csv /read_table function arguments

# ARGUMENT              DESCRIPTION
# path                  String indicating filesystem location, URL, or file-like object
# sep or delimiter      Character sequence or regular expression to use to split fields in each row
# header                Row number to use as column names. Defaults to 0 (first row), but should be None if there is no header row
# index_col             Column numbers or names to use as the row index in the result. Can be a single name/number or a list of them for a hierarchical index
# names                 List of column names for result, combine with header=None
# skiprows              Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip
# na_values             Sequence of values to replace with NA
# comment               Character or characters to split comments off the end of lines
# parse_dates           Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (for example if date/time split across two columns)
# keep_date_col         If joining columns to parse date, drop the joined columns. Default True
# converters            Dict containing column number of name mapping to functions. For example {'foo': f} would apply the function f to all values in the 'foo' column
# dayfirst              When parsing potentially ambiguous dates, treat as international format (e.g. 7/6/2012 -> June 7, 2012). Default False
# date_parser           Function to use to parse dates
# nrows                 Number of rows to read from beginning of file
# iterator              Return a TextParser object for reading file piecemeal
# chunksize             For iteration, size of file chunks
# skip_footer           Number of lines to ignore at end of file
# verbose               Print various parser output information, like the number of missing values placed in non-numeric columns
# encoding              Text encoding for unicode. For example 'utf-8' for UTF-8 encoded text
# squeeze               If the parsed data only contains one column return a Series
# thousands             Separator for thousands, e.g. ',' or '.'

### Reading Text Files in Pieces

When **processing very large files** or figuring out the right set of arguments to correctly
process a large file, you may only want to read in a small piece of a file or **iterate through
smaller chunks of the file**.

In [260]:
result = pd.read_csv('C:/srdev/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [261]:
# If you want to only read out a small number of rows (avoiding reading the entire file), specify that with nrows:
pd.read_csv('C:/srdev/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [94]:
# To read out a file in pieces, specify a chunksize as a number of rows:
chunker = pd.read_csv('C:/srdev/ex6.csv', chunksize=1000)
chunker

In [268]:
# The TextParser object returned by read_csv allows you to iterate over the parts of the
# file according to the chunksize. For example, we can iterate over ex6.csv, aggregating
# the value counts in the 'key' column like so:

chunker = pd.read_csv('C:/srdev/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

C:\ProgramData\Anaconda3-2020.02\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


In [270]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data Out to Text Format

In [272]:
# Data can also be exported to delimited format. Let’s consider one of the CSV files read above:
data = pd.read_csv('C:/srdev/ex5.csv')
data 

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [273]:
# Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:
data.to_csv('C:/srdev/out.csv')

In [274]:
!cat C:/srdev/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [279]:
# Other delimiters can be used, of course (writing to sys.stdout so it just prints the text result): 
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [280]:
# Missing values appear as empty strings in the output. You might want to denote them
# by some other sentinel value:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [281]:
# With no other options specified, both the row and column labels are written. Both of these can be disabled:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [283]:
# You can also write only a subset of the columns, and in an order of your choosing
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### JSON Data

**JSON (short for JavaScript Object Notation)** has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It is
a much more flexible data format than a tabular text form like CSV. Here is an example:

In [286]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [287]:
import json

In [288]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

In [289]:
# json.dumps on the other hand converts a Python object back to JSON:
asjson = json.dumps(result)

### Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and
XML formats. lxml (http://lxml.de) is one that has consistently strong performance in
parsing very large files. lxml has multiple programmer interfaces; first I’ll show using
lxml.html for HTML, then parse some XML using lxml.objectify.
Many websites make data available in HTML tables for viewing in a browser, but not
downloadable as an easily machine-readable format like JSON, HTML, or XML. I noticed
that this was the case with Yahoo! Finance’s stock options data. If you aren’t
familiar with this data; options are derivative contracts giving you the right to buy
(call option) or sell (put option) a company’s stock at some particular price (the
strike) between now and some fixed point in the future (the expiry). People trade both
call and put options across many strikes and expiries; this data can all be found together
in tables on Yahoo! Finance.

In [117]:
import urllib
import urllib.request
from lxml.html import parse

In [118]:
page = urllib.request.urlopen('http://www.python.org/')
print(page.read())

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

#### Articles on web scrapingn with Python

[Beginner’s guide to Web Scraping in Python using BeautifulSoup](https://www.analyticsvidhya.com/blog/2015/10/beginner-guide-web-scraping-beautiful-soup-python/)

[How to Web Scrape with Python in 4 Minutes](https://towardsdatascience.com/how-to-web-scrape-with-python-in-4-minutes-bc49186a8460)

[A beginner's guide to web scraping with Python](https://opensource.com/article/20/5/web-scraping-python)

### Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files using
the ExcelFile class. Interally ExcelFile uses the xlrd and openpyxl packages, so you
may have to install them first. To use ExcelFile, create an instance by passing a path
to an xls or xlsx file:

In [25]:
xls_file = pd.ExcelFile('C:\srdev\Churn.xlsx')
# Data stored in a sheet can then be read into DataFrame using parse:
table = xls_file.parse('Iris2')

In [122]:
table

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [124]:
df = pd.DataFrame(table)
df.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Interacting with Databases
In many applications data rarely comes from text files, that being a fairly inefficient
way to store large amounts of data. **SQL-based relational databases (such as SQL Server,
PostgreSQL, and MySQL)** are in wide use, and many alternative **non-SQL (so-called
NoSQL) databases have become quite popular**. The choice of database is usually dependent
on the performance, data integrity, and scalability needs of an application.
Loading data from SQL into a DataFrame is fairly straightforward, and pandas has
some functions to simplify the process.

In [31]:
import sqlite3

In [125]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);"""

In [126]:
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [34]:
# Then, insert a few rows of data:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [35]:
cursor = con.execute('select * from test')

In [36]:
rows = cursor.fetchall()

In [37]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

This is quite a bit of munging that you’d rather not repeat each time you query the
database. pandas has a read_frame function in its pandas.io.sql module that simplifies
the process. Just pass the select statement and the connection object:

In [41]:
import pandas.io.sql as sql

sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## Data Wrangling: Clean, Transform, Merge, Reshape

### Combining and Merging Data Sets

Data contained in pandas objects can be combined together in a number of built-in
ways:
+ **pandas.merge** connects rows in DataFrames based on one or more keys. This will
be familiar to users of SQL or other relational databases, as it implements database
join operations.
+ **pandas.concat** glues or stacks together objects along an axis.

### Database-style DataFrame Merges

**Merge or join operations combine data sets by linking rows using one or more keys**.
These operations are central to relational databases. The merge function in pandas is
the main entry point for using these algorithms on your data.

In [99]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})

df1;df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a **many-to-one merge situation**; the data in df1 has multiple rows
labeled a and b, whereas df2 has only one row for each value in the key column. Calling
merge with these objects we obtain:

In [45]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that **I didn’t specify which column to join on. If not specified, merge uses the
overlapping column names as the keys**. It’s a good practice to specify explicitly, though:

In [46]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the **column names are different** in each object, you can specify them separately:

In [100]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})

df3;df4

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


You probably noticed that the 'c' and 'd' values and associated data are missing from
the result. **By default merge does an 'inner' join**; the keys in the result are the intersection.
Other possible options are **'left', 'right', and 'outer'.** The outer join takes the
union of the keys, combining the effect of applying both left and right joins:

In [51]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


**Many-to-many merges have well-defined though not necessarily intuitive behavior.**
Here’s an example:

In [101]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [102]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-many joins form the Cartesian product of the rows. Since there were 3 'b'
rows in the left DataFrame and 2 in the right one, there are 6 'b' rows in the result.
The join method only affects the distinct key values appearing in the result:

In [55]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


**To merge with multiple keys**, pass a list of column names

In [58]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


A last issue to consider in merge operations is the **treatment of overlapping column names**.

In [59]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [60]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index
In some cases, the merge key or keys in a DataFrame will be found in its index. In this
case, you can pass left_index=True or right_index=True (or both) to indicate that the
index should be used as the merge key:

In [63]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

left1; right1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [64]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Since the **default merge method is to intersect the join keys**, you can instead form the
union of them with an outer join:

In [65]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### Concatenating Along an Axis
Another kind of data combination operation is alternatively referred to as concatenation,
binding, or stacking. NumPy has a concatenate function for doing this with raw
NumPy arrays:

In [66]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [67]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In the context of pandas objects such as Series and DataFrame, having labeled axes
enable you to further generalize array concatenation. In particular, you have a number
of additional things to think about:
+ If the objects are indexed differently on the other axes, should the collection of
axes be unioned or intersected?
+ Do the groups need to be identifiable in the resulting object?
+ Does the concatenation axis matter at all?
The concat function in pandas provides a consistent way to address each of these concerns.
I’ll give a number of examples to illustrate how it works. Suppose we have three
Series with no index overlap:

In [103]:
s1 = pd.Series([0, 1], index=['a', 'b'])

s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

s3 = pd.Series([5, 6], index=['f', 'g'])

s1;s2;s3

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [71]:
# Calling concat with these object in a list glues together the values and indexes:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

**By default concat works along axis=0**, producing another Series. If you pass axis=1, the
result will instead be a DataFrame (axis=1 is the columns):

In [73]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted
union (the 'outer' join) of the indexes. **You can instead intersect them by passing
join='inner':**

In [104]:
s4 = pd.concat([s1 * 5, s3])
s4 

a    0
b    5
f    5
g    6
dtype: int64

In [76]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [77]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


### Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an
example:

In [82]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,  'k2': [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [84]:
# The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate or not:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [83]:
# Relatedly, drop_duplicates returns a DataFrame where the duplicated array is True: 
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


Both of these methods by **default consider all of the columns**; alternatively you can
**specify any subset of them to detect duplicates**. Suppose we had an additional column
of values and wanted to filter duplicates only based on the 'k1' column:

In [86]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [87]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3
